# Sesión 5 de Stata

Estoy probando esta nueva interacción de *Jupyter Notebooks* con *Stata* para poder trabajar de manera más cómoda (como un ser humano). 

La idea de esta sesión es trabajar dos tipo de transformación que cambian la naturaleza misma de la tabla al reacomodar su estructura:  `collapse` y `reshape`. El uso de estos comandos cambia **por completo** su tabla (base de datos).

## Setup

Como siempre, haremos un *do file* con su respectivo setup y nuestro (ahora obsoleto) *log*.

In [1]:
*=====================================================================

// PROYECTO: Quinta introductoria a STATA
// PROGRAMA: sesion_5.do
// TAREA: De long a wide
// CREADO POR: Manuel Toral
// FECHA DE CREACIÓN: 2022-07-28
// FECHA DE ACTUALIZACIÓN: 2022-07-28

*========================================================================
cd "C:\Users\User\Documents\MUCD_stata_intro\Sesion_5"
log using "logs_notebooks\log_sesion5.smcl", text replace name(sesion5)


C:\Users\User\Documents\MUCD_stata_intro\Sesion_5

--------------------------------------------------------------------------------
      name:  sesion5
       log:  C:\Users\User\Documents\MUCD_stata_intro\Sesion_5\logs_notebooks\lo
> g_sesion5.smcl
  log type:  text
 opened on:  29 Jul 2022, 10:21:22


# Datos

Vamos a usar los datos de incidencia delictiva del Secretariado, para no perder la costumbre.

In [18]:
// 1. Datos
import delimited "data\IDEFC_NM_jun22.csv", clear

(19 vars, 25,088 obs)


Ante de continuar, podemos observar cómo la tabla tiene una estructura *horizontal* (*wide*) que tiene columnas con información *del mismo tipo* repartidas por varias variables. Es decir, si vemos la primer fila, las variables `enero` y `febrero` contienen números que son de la misma naturaleza y son consistentes entre sí: el número de homicidios dolosos con arma de fuego.

In [3]:
list in 1


     +---------------------------------------------------------------------+
  1. |  año | clave_~t |        entidad |             bienjurídicoafectado |
     | 2015 |        1 | Aguascalientes | La vida y la Integridad corporal |
     |---------------------------------------------------------------------|
     | tipoded~o |  subtipodedelito |         modalidad | enero | febrero  |
     | Homicidio | Homicidio doloso | Con arma de fuego |     3 |       0  |
     |------------------------------+---------------------------+----------|
     | marzo | abril | mayo | junio | julio | agosto | septie~e | octubre  |
     |     2 |     1 |    1 |     1 |     2 |      1 |        2 |       2  |
     |---------------------------------------------------------------------|
     |             noviem~e             |             diciem~e             |
     |                    2             |                    1             |
     +---------------------------------------------------------------------

Y esta estructura está muy bien para cuando necesitamos estudiar un tipo de fenómeno, pero justo una de las ventajas de la tabla de incidencia delictiva es que nos permite hacer **comparaciones en el tiempo**. Por lo que el reto es hacer *una sola variable* que agrupe los números comparables.

# Colapsar

Pero antes, para facilitarnos la vida, vamos a hacer un colapso. Esto nos va a permitir utilizar la información que necesitamos y no más. La idea de "colapsar" es agregar. Es decir, como su nombre lo indica, colapsar la estructura de nuestra tabla en formas más simples a través de operaciones aritméticas. 

Vamos a resolver esta pregunta, una que casi nadie se hace en México: *¿Cómo se comportan los homicidios dolosos con arma de fuero en el tiempo?*

Como pueden ver en la documentación, `collapse` tiene una estructura como esta:

> `collapse variable1 variable2 by(grupo)`

En español podría ser: *colapsa la estrctura de esta tabla con base en la "variable 1" y la "variable 2", pero conserva los grupos de la variable "grupo"*.

Entonces, la instrucción para resolver la pregunta que acabamos de plantear tendría que ser de la siguiente forma (en español):
1. *conserva sólo aquellas observaciones que se tengan como subtipo de delito "Homicidio doloso" y como modalidad "con arma de fuego".*
2. *colapsa la estrctura de IDEFC_NM_jun22 con base en las variables de "enero" a "diciembre", pero conserva los grupos de la variable "año"*. 

In [19]:
// 2. Transformaciones y limpieza


// 2.1 Conserva sólo el subtipo de delito y modalidad que definen "Homicidio doloso con arema de fuego"

keep if subtipodedelito == "Homicidio doloso" & modalidad == "Con arma de fuego"

d
tab modalidad
tab subtipodedelito






Contains data
  obs:           256                          
 vars:            19                          
--------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------
año             int     %8.0g                 Año
clave_ent       byte    %8.0g                 Clave_Ent
entidad         str31   %31s                  Entidad
bienjurídicoa~o str52   %52s                  Bien jurídico afectado
tipodedelito    str78   %78s                  Tipo de delito
subtipodedelito str78   %78s                  Subtipo de delito
modalidad       str84   %84s                  Modalidad
enero           int     %8.0g                 Enero
febrero         int     %8.0g                 Febrero
marzo           int     %8.0g                 Marzo
abril           int     %8.0g                 Abril
mayo

In [20]:
collapse (sum) enero-diciembre, by(año)
d
list




Contains data
  obs:             8                          
 vars:            13                          
--------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------
año             int     %8.0g                 Año
enero           double  %8.0g                 (sum) enero
febrero         double  %8.0g                 (sum) febrero
marzo           double  %8.0g                 (sum) marzo
abril           double  %8.0g                 (sum) abril
mayo            double  %8.0g                 (sum) mayo
junio           double  %8.0g                 (sum) junio
julio           double  %8.0g                 (sum) julio
agosto          double  %8.0g                 (sum) agosto
septiembre      double  %8.0g                 (sum) septiembre
octubre         double  %8.0g                 

In [21]:
// 3. Convertir de wide a long

rename (enero-diciembre) (homicidios(#)) ,addnumber

gen id = _n

reshape long homicidios, i(id) j(mes)

d

//reshape wide homicidios, i(id) j(mes)

//d





(note: j = 1 2 3 4 5 6 7 8 9 10 11 12)


-----------------------------------------------------------------------------
Number of obs.                        8   ->      96
Number of variables                  14   ->       4
j variable (12 values)                    ->   mes
xij variables:

-----------------------------------------------------------------------------


Contains data
  obs:            96                          
 vars:             4                          
--------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------
id              float   %9.0g                 
mes             byte    %9.0g                 
año             int     %8.0g                 Año
homicidios      double  %8.0g                 
--------------------------------------------------------------

In [23]:
gen dia = 1

variable dia already defined


r(110);


In [ ]:
egen fecha = concat(mes, "1" )

In [ ]:
%browse

In [7]:
log close sesion5

      name:  sesion5
       log:  C:\Users\User\Documents\MUCD_stata_intro\Sesion_5\logs_notebooks\lo
> g_sesion5.smcl
  log type:  text
 closed on:  29 Jul 2022, 10:07:10
--------------------------------------------------------------------------------
